In [10]:
import pickle
import torch
import io
from torchvision import models
from torchinfo import summary

from pytorch_quantization import quant_modules
quant_modules.initialize()

from src.quantization.quantized_model import QuantizedModel
from src.exploration.problems import LayerwiseQuantizationProblem

path = 'retrained_model.m'

m = models.resnet.resnet50()

summary(m, (1, 3, 224, 224))








Layer (type:depth-idx)                             Output Shape              Param #
ResNet                                             [1, 1000]                 --
├─QuantConv2d: 1-1                                 [1, 64, 112, 112]         9,408
│    └─TensorQuantizer: 2-1                        [1, 3, 224, 224]          --
│    └─TensorQuantizer: 2-2                        [64, 3, 7, 7]             --
├─BatchNorm2d: 1-2                                 [1, 64, 112, 112]         128
├─ReLU: 1-3                                        [1, 64, 112, 112]         --
├─MaxPool2d: 1-4                                   [1, 64, 56, 56]           --
├─Sequential: 1-5                                  [1, 256, 56, 56]          --
│    └─Bottleneck: 2-3                             [1, 256, 56, 56]          --
│    │    └─QuantConv2d: 3-1                       [1, 64, 56, 56]           4,096
│    │    └─BatchNorm2d: 3-2                       [1, 64, 56, 56]           128
│    │    └─ReLU: 3-3      

In [11]:
m.load_state_dict(torch.load(path))

<All keys matched successfully>

In [30]:
for name, module in m.named_modules():
    if isinstance(module, quant_modules.quant_nn.TensorQuantizer):
        module.enable_quant()

In [31]:
for name, module in m.named_modules():
    if isinstance(module, quant_modules.quant_nn.TensorQuantizer):
        print("{}: {} bits, mod: {}".format(name, module.num_bits, module))

conv1._input_quantizer: 8 bits, mod: TensorQuantizer(8bit fake per-tensor amax=2.6387 calibrator=MaxCalibrator scale=1.0 quant)
conv1._weight_quantizer: 8 bits, mod: TensorQuantizer(8bit fake axis=0 amax=[0.2427, 1.9790](64) calibrator=MaxCalibrator scale=1.0 quant)
layer1.0.conv1._input_quantizer: 8 bits, mod: TensorQuantizer(8bit fake per-tensor amax=40.8780 calibrator=MaxCalibrator scale=1.0 quant)
layer1.0.conv1._weight_quantizer: 8 bits, mod: TensorQuantizer(8bit fake axis=0 amax=[0.2322, 1.5130](64) calibrator=MaxCalibrator scale=1.0 quant)
layer1.0.conv2._input_quantizer: 8 bits, mod: TensorQuantizer(8bit fake per-tensor amax=17.1855 calibrator=MaxCalibrator scale=1.0 quant)
layer1.0.conv2._weight_quantizer: 8 bits, mod: TensorQuantizer(8bit fake axis=0 amax=[0.1967, 1.4748](64) calibrator=MaxCalibrator scale=1.0 quant)
layer1.0.conv3._input_quantizer: 8 bits, mod: TensorQuantizer(8bit fake per-tensor amax=23.5362 calibrator=MaxCalibrator scale=1.0 quant)
layer1.0.conv3._weight_

In [ ]:
m.conv1.weight[0][0]

tensor([[-0.0095, -0.0046,  0.0364, -0.0647,  0.0478, -0.0206,  0.0083],
        [-0.0567,  0.0460,  0.0790, -0.1770,  0.0911,  0.0316, -0.0560],
        [ 0.0694, -0.2692,  0.4052, -0.1330, -0.1630,  0.2209, -0.0708],
        [ 0.0244,  0.1294, -0.6641,  1.1666, -0.9223,  0.2885, -0.0361],
        [-0.1096,  0.3817, -0.4542, -0.1022,  0.6848, -0.5775,  0.2255],
        [ 0.0242, -0.1777,  0.6440, -0.9113,  0.5268, -0.0485, -0.0679],
        [ 0.0439, -0.1317,  0.0174,  0.2513, -0.3571,  0.1905, -0.0224]],
       device='cuda:0', grad_fn=<SelectBackward0>)

: 

In [32]:
print(m.conv1.weight_quantizer)

TensorQuantizer(8bit fake axis=0 amax=[0.2427, 1.9790](64) calibrator=MaxCalibrator scale=1.0 quant)
